In [1]:
import sys
sys.path.append('..')
sys.path.append('../hypothesis_generation')


from PIL import Image as PImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import pickle
import json
from torchvision.utils import make_grid
import torch
from hypothesis_generation.prefix_postfix import PrefixPostfix
from hypothesis_generator import GrammarExpander
from reduce_and_process_hypotheses import MetaDatasetExample
from reduce_and_process_hypotheses import HypothesisEval

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
properties_file = "/private/home/ramav/code/ad_hoc_categories/concept_data/clevr_typed_fol_properties.json"
grammar_expander = ("/private/home/ramav/code/ad_hoc_categories"
                    "/concept_data/temp_data/v2_typed_simple_fol_clevr_typed_fol_properties.pkl")
program_converter = PrefixPostfix(
    properties_file, grammar_expander_file=grammar_expander
)

In [3]:
def visualize_example(query_and_support, data_idx, split_name="train", nrows=5, filter_or=True,
                      nimages_per_row=5):
    # TODO(ramav): nimages per row is not something that should be left as a free variable to supply.        
    _TOTAL_NEGATIVES=20
        
    hypothesis = query_and_support[split_name].hypothesis
    positive_images_per_row = 5
    
    fig = plt.figure(figsize=(10, 20))
    for idx, image_path in enumerate(query_and_support[split_name].image_paths):
        ax = plt.subplot(1, positive_images_per_row, idx + 1)
        img = mpimg.imread(image_path)
        plt.imshow(img)
        plt.axis('off')
        if idx == 0:
            ax.set_title("%d] <Original Query>: %s" % (data_idx, program_converter.postfix_to_prefix(hypothesis)))
    
    
    if filter_or == True:
        idx_to_print = [idx for idx, x in enumerate(
            query_and_support[split_name].alternate_hypothesis_str) if " or " not in x]
        
    else:
        idx_to_print = range(len(query_and_support[split_name].alternate_hypothesis_str))
        
    if len(query_and_support[split_name].alternate_hypothesis_str) * _TOTAL_NEGATIVES !=  len(
        query_and_support[split_name].image_paths_negative):
        print("%d ratio" % (len(query_and_support[split_name].image_paths_negative)
                           / len(query_and_support[split_name].alternate_hypothesis_str)))
        raise ValueError
        
    printed_idx_count = 0
    nrows = len(idx_to_print)
    fig = plt.figure(figsize=(10, 4 + 2 * nrows))    

    
    for idx, image_path in enumerate(query_and_support[split_name].image_paths_negative):
        if (int(idx/_TOTAL_NEGATIVES) not in idx_to_print) or (idx % _TOTAL_NEGATIVES >= nimages_per_row):
            continue
            
        ax = plt.subplot(nrows, nimages_per_row, printed_idx_count + 1)
        img = mpimg.imread(image_path)
        plt.imshow(img)
        plt.axis('off')
        if printed_idx_count % nimages_per_row == 0:
            ax.set_title("%d] <Alternate>: %s" % (data_idx, program_converter.postfix_to_prefix(
                    query_and_support[split_name].alternate_hypothesis_str[int(idx/_TOTAL_NEGATIVES)])))
            
        printed_idx_count += 1

In [6]:
dataset_json = ("${CURI_DATA_PATH}/"
                "hypotheses/v2_typed_simple_fol_depth_6_trials_2000000_ban_1_max_scene_id_200/"
                "comp_sampling_log_linear_test_threshold_0.10_pos_im_5_neg_im_20_train_examples_"
                "500000_neg_type_alternate_hypotheses_alternate_hypo_1_random_seed_42.pkl")

In [7]:
with open(dataset_json, 'rb') as f:
    dataset = pickle.load(f)
    dataset = dataset["meta_dataset"]

### Qualitative Results ###

NOTE: Only showing alternate hypotheses for which there is no "or" clause, so that we are able to look at and focus on that subset. Location origin (0, 0) is at the top left corner, and the image is the bottom right quadrant.

#### List of properties present in the dataset ####
       
"COUNTS": [1, 2, 3],

"COLOR": ["gray",
        "red",
        "blue",
        "green",
        "brown",
        "purple",
        "cyan",
        "yellow"]
        
"SHAPE": 
    [
        "cube",
        "sphere",
        "cylinder"
    ],
    
"MATERIAL": 
    [
        "rubber",
        "metal"
    ],
    
"SIZE":
    [
        "large",
        "small"
    ],
    
"LOCX":
    [
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8"
    ],
"LOCY":
    [
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8"
    ],
    
#### Format ####
There is example number < original query> followed by images for that concept, and then example number <alternate> followed by alternate images for the other concepts which explain the images corresponding to the <original query>

In [13]:
for this_idx, datum in enumerate(dataset):
    if this_idx > 10:
        break
    visualize_example(datum, this_idx, "support")

AttributeError: 'MetaDatasetExample' object has no attribute 'image_paths_positive'

<Figure size 720x1440 with 0 Axes>